# RoboCall Mitigation Database
- Original file: rmd-original.csv
    - Contains details of thousands of providers
    - Details inculde business information such as FRN, Company Name, Contact Details, Person responsible for S/S deployment
    - Detials also include the level of S/S implementation - Yes / No / Partial
- Task of this Jupyter notebook is to filter out duplicates and NULL entries and extract US providers' S/S implementation details 

In [60]:
# Imports
import pandas as pd
import os

### This section contains functions that are used to manipulate the CSV
- add_index_column
- extract_columns
- remove_newlines_inside_rows
- remove_duplicates_based_on_2_fields
- filter_by_country
- remove_null_rows
- update_implementation_field
- sort_by_business_name

In [64]:
### Begin add_index_column
# Takes a CSV file and inserts an index column at the beginning and outputs a new CSV file
def add_index_column(input_file, output_file):
    try:
        df = pd.read_csv(input_file, dtype=str)
        df.insert(0, 'index', range(1, len(df) + 1))
        df.to_csv(output_file, index=False)

        print(f"Indexed CSV saved to {output_file}")
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End add_index_column

### Begin extract_columns
# Takes a CSV file and column names as input an outputs a new CSV file containing only those columns specified
def extract_columns(input_file, output_file, fields):
    try:
        df = pd.read_csv(input_file, usecols=fields, dtype=str)
        df.to_csv(output_file, index=False)

        print(f"Extracted columns  CSV saved to {output_file}")
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End extract_columns

### Begin remove_newlines_inside_rows
# Takes a CSV file and column names as input and removes any new lines present in any of the column entries. Outputs a new CSV file.
def remove_newlines_inside_rows(file_path, columns_to_process, output_path):
    try:
        # Load the CSV file
        df = pd.read_csv(file_path, dtype=str)

        # Function to replace newlines within double quotes
        def replace_newlines_in_quotes(text):
            try:
                text = text.replace("\n", ' ')
                text = text.replace("\r", ' ')
            except:
                None
            return text

        # Process the specified columns
        for column in columns_to_process:
            if column in df.columns:
                df[column] = df[column].apply(replace_newlines_in_quotes)

        # Save the updated CSV
        df.to_csv(output_path, index=False)

        print(f"No newlines CSV saved to {output_path}")

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End remove_newlines_inside_rows

### Begin remove_duplicates_based_on_2_fields
# Reads a CSV file, removes duplicates based on two specified fields, and writes the result to a new file.
def remove_duplicates_based_on_2_fields(csv_file, field1, field2, output_file):
    # Load the CSV file
    try:
        # Load the CSV file
        df = pd.read_csv(csv_file)
        
        # Separate rows with NaN in the specified fields
        nan_rows = df[df[[field1, field2]].isna().any(axis=1)]
        
        # Identify rows without NaN in the specified fields
        valid_rows = df.dropna(subset=[field1, field2])
        
        # Identify duplicates in valid rows
        duplicates = valid_rows.duplicated(subset=[field1, field2], keep=False)
        
        # Filter unique rows from valid rows (first occurrence of duplicates + rows that are not duplicates)
        deduplicated_valid_rows = valid_rows[~duplicates | (valid_rows.duplicated(subset=[field1, field2], keep='first'))]
        
        # Combine deduplicated valid rows with rows that have NaN
        final_df = pd.concat([deduplicated_valid_rows, nan_rows], ignore_index=True)
        
        # Save the result to a new CSV file
        final_df.to_csv(output_file, index=False)
        
        print(f"Deduplicated data based on {field1} and {field2} has been saved to {output_file}.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End remove_duplicates_based_on_2_fields

### Begin filter_by_country
# Extract out providers based on country
def filter_by_country(input_file, country, output_file):
    try:
        df = pd.read_csv(input_file, dtype=str)
        filtered_df = df[df['country'] == country]
        filtered_df.to_csv(output_file, index=False)

        print(f"Country filtered CSV saved to {output_file}")
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End filter_by_country

### Begin remove_null_rows
# If any of the fields provided are null then, that row is deleted
def remove_null_rows(input_file, fields, output_file):
    try:
        df = pd.read_csv(input_file, dtype=str)
        filtered_df = df.dropna(subset=fields)
        filtered_df.to_csv(output_file, index=False)

        print(f"Removed null rows CSV saved to {output_file}")
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End remove_null_rows

### Begin update_implementation_field
# Update the implementation field according to the map
def update_implementation_field(input_file, output_file):
    mapping = {
        "Complete STIR/SHAKEN Implementation": "Yes",
        "Partial STIR/SHAKEN Implementation - Performing Robocall Mitigation": "Partial",
        "No STIR/SHAKEN Implementation - Performing Robocall Mitigation": "No"
    }
    
    try:
        df = pd.read_csv(input_file, dtype=str)
        df['implementation'] = df['implementation'].map(mapping).fillna(df['implementation'])
        df.to_csv(output_file, index=False)

        print(f"Updated implementation field CSV saved to {output_file}")
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End update_implementation_field

### Begin sort_by_business_name
# Sort alphabetically based on business_name
def sort_by_business_name(input_file, output_file):
    try:
        df = pd.read_csv(input_file, dtype=str)
        df = df.sort_values(by="business_name", ascending=True)
        df.to_csv(output_file, index=False)

        print(f"Sorted CSV saved to {output_file}")
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End sort_by_business_name


### This section contains functions that are used to display the requested field from the CSV
- print_duplicated_rows
- print_duplicated_rows_2_fields
- print_stats

In [68]:
### Begin print_duplicated_rows
# Takes a CSV file as input and prints duplicates in each column
def print_duplicated_rows(file_path):
    try:
        # Load the CSV file
        df = pd.read_csv(file_path, dtype=str)

        # Identify duplicates in each column
        for column in df.columns:
            duplicates = df[column][df[column].duplicated()]
            if not duplicates.empty:
                print(f"Duplicates in column '{column}':")
                print(duplicates)
                print()
            else:
                print(f"No duplicates found in column '{column}'.")

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End print_duplicated_rows

### Begin print_duplicated_rows_2_fields
# takes a CSV file and two field names as input, then selects and prints rows where both field entries are duplicated
def print_duplicated_rows_2_fields(input_csv, field1, field2):
    # Load the CSV file
    try:
        df = pd.read_csv(input_csv)
    except FileNotFoundError:
        print(f"Error: The file '{input_csv}' was not found.")
        return
    except Exception as e:
        print(f"Error reading the file: {e}")
        return

    # Check if the specified fields exist
    if field1 not in df.columns or field2 not in df.columns:
        print(f"Error: One or both fields '{field1}' and '{field2}' do not exist in the CSV file.")
        return

    # Identify rows where both fields have duplicate values
    duplicated_rows = df[df.duplicated(subset=[field1, field2], keep=False)]
    
    # Group by the specified fields and sort the results
    grouped_duplicates = duplicated_rows.sort_values(by=[field1, field2])
    
    # Print the grouped duplicate rows (only the specified fields)
    if grouped_duplicates.empty:
        print("No rows with duplicate values in both fields were found.")
    else:
        print("Grouped duplicate rows with specified fields:")
        print(grouped_duplicates[['index', field1, field2]].to_string(index=False))
### End print_duplicated_rows_2_fields

### Begin 
# Print stats of the final CSV
def print_stats(input_csv):
    try:
        df = pd.read_csv(input_csv, dtype=str)

        count_total_providers = df.shape[0]

        count_impl_yes = df["implementation"].eq("Yes").sum()
        count_impl_partial = df["implementation"].eq("Partial").sum()
        count_impl_no = df["implementation"].eq("No").sum()

        count_vo_pwd = df["voice_service_provider_choice"].eq("Yes").sum()
        count_gw_pwd = df["gateway_provider_choice"].eq("Yes").sum()
        count_int_pwd = df["intermediate_provider_choice"].eq("Yes").sum()

        print("CSV stats:")
        print(f"Total providers = {count_total_providers}")

        print(f"Full S/S implementation = {count_impl_yes}")
        print(f"Partial S/S implementation = {count_impl_partial}")
        print(f"No S/S implementation = {count_impl_no}")

        print(f"Count of voice providers = {count_gw_pwd}")
        print(f"Count of gateway providers = {count_vo_pwd}")
        print(f"Count of intermediate providers = {count_int_pwd}")


    except FileNotFoundError:
        print(f"Error: The file '{input_csv}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
### End

### Main

In [65]:
# Filenames
og_csv = "csv/rmd-original.csv"
indexed_csv = "csv/rmd-1-indexed.csv"
nonl_csv = "csv/rmd-2-nonl.csv"
country_csv = "csv/rmd-3-country.csv"
nonull_csv = "csv/rmd-4-nonull.csv"
upd_impl_csv = "csv/rmd-5-updimpl.csv"
dedup_2F_csv = "csv/rmd-6-dedup2F.csv"
sort_csv = "csv/rmd-7-sort.csv"
final_csv = "csv/rmd-final.csv"

# Func args
nonl_cols = ["business_address", "other_dba_names", "previous_dba_names", "contact_business_address", "other_frns"]
country = "United States of America"
nonull_cols = ["implementation", "frn", "business_name"]

# Adding index field to the original csv
add_index_column(og_csv, indexed_csv)

# Remove new lines in business_address and contact_business_address fields
remove_newlines_inside_rows(indexed_csv, nonl_cols, nonl_csv)

# Filter by country
filter_by_country(nonl_csv, country, country_csv)

# Remove null rows
remove_null_rows(country_csv, nonull_cols, nonull_csv)

# Update Implementation field
update_implementation_field(nonull_csv, upd_impl_csv)

# remove rows where
# 1. business_name and business_address are same
# 2. business_name and contact_business_address are same
# 3. business_name and are frn same
remove_duplicates_based_on_2_fields(upd_impl_csv, "business_name", "business_address", dedup_2F_csv)
remove_duplicates_based_on_2_fields(dedup_2F_csv, "business_name", "contact_business_address", dedup_2F_csv)
remove_duplicates_based_on_2_fields(dedup_2F_csv, "business_name", "frn", dedup_2F_csv)

# sort rows based on business name
sort_by_business_name(dedup_2F_csv, sort_csv)

Indexed CSV saved to csv/rmd-1-indexed.csv
No newlines CSV saved to csv/rmd-2-nonl.csv
Country filtered CSV saved to csv/rmd-3-country.csv
Removed null rows CSV saved to csv/rmd-4-nonull.csv
Updated implementation field CSV saved to csv/rmd-5-updimpl.csv
Deduplicated data based on business_name and business_address has been saved to csv/rmd-6-dedup2F.csv.
Deduplicated data based on business_name and contact_business_address has been saved to csv/rmd-6-dedup2F.csv.
Deduplicated data based on business_name and frn has been saved to csv/rmd-6-dedup2F.csv.
Sorted CSV saved to csv/rmd-7-sort.csv


### Delete final file if it exists and write a new one

In [66]:
if os.path.exists(final_csv):
    os.remove(final_csv)
os.rename(sort_csv, final_csv)

In [69]:
print_stats(final_csv)

CSV stats:
Total providers = 7346
Full S/S implementation = 4111
Partial S/S implementation = 1453
No S/S implementation = 1782
Count of voice providers = 420
Count of gateway providers = 6720
Count of intermediate providers = 681
